In [1]:

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, load_metric
import evaluate
from preprocessing import preprocessing
from transformers import DataCollatorForTokenClassification
import numpy as np


/opt/homebrew/Caskroom/miniconda/base/envs/anlpa3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/Users/martincarrasco/Documents/VU/Y1P4/ANLP/AdvancedNLP-VU/A3/utils.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/opt/homebrew/Caskroom/miniconda/base/envs/anlpa3/lib/python3.11/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository c

In [3]:
df = preprocessing('en_ewt-up-train.conllu')
label_list = list(df['label'].unique())

In [ ]:
sent_df = df.groupby(['sentence_id']).agg(lambda x: x.tolist())

In [ ]:
ds = Dataset.from_pandas(sent_df)

In [40]:
ds['label']

['O,O,O,O,O,ARG0,V,ARG1,O,O,O,O,O,O,O,O,O,ARGM-LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ARG0,V,ARG1,O,O,O,O,O,O,O,O,O,ARGM-LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ARG0,V,ARG1,O,O,O,O,O,O,O,O,O,ARGM-LOC,O,O,O,O,O,O,O,O,O,O,O',
 'O,O,V,O,O,O,ARG1,O,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,ARG1,O,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,ARG1,O,O,O,O,O,O,O,O,O,O,O',
 'O,O,O,ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O,O,O,O,O,ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O,O,O,O,O,ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O,O',
 'O,O,O,V,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,O,O,O,O,O,O,O,O,O',
 'O,ARG1,O,O,V,ARG2,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ARG1,O,O,V,ARG2,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ARG1,O,O,V,ARG2,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O',
 'O,O,V,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,O,O,O,O,O,O,O,O,O,V,O,O,O,O,O,O,O,O,O,O',
 'ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O,ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O,ARG0,V,O,O,O,ARG1,O,O,O,O,O,O,O',
 'O,O,V,O,O,O,O,O,

In [14]:
from utils import tokenize_and_align_labels
tokenized_datasets = ds.map(lambda x: tokenize_and_align_labels(tokenizer, x))

Map:   0%|          | 0/40482 [00:00<?, ? examples/s]


IndexError: string index out of range

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
SEP_TOKEN_ID = tokenizer.all_special_ids[tokenizer.all_special_tokens.index('[SEP]')]

In [47]:
row = sent_df.iloc[0]
predicates = list(set(row['predicate']))
for pred in predicates:
    if pred == '_':
        continue
    pred = pred.split('.')[0]
    tokenized_input = tokenizer(row['token'], is_split_into_words=True)
    pred_tokens = tokenizer([pred], is_split_into_words=True)
    for idx in range(len(pred_tokens["input_ids"])-1):
        idx = idx+1
        tokenized_input['input_ids'].append(pred_tokens["input_ids"][idx])
        tokenized_input['token_type_ids'].append(pred_tokens["token_type_ids"][idx])
        tokenized_input['attention_mask'].append(pred_tokens["attention_mask"][idx])

{'input_ids': [101, 3102, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'al', '-', 'za', '##man', ':', 'american', 'forces', 'killed', 'sha', '##ikh', 'abdullah', 'al', '-', 'an', '##i', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'q', '##ai', '##m', ',', 'near', 'the', 'syrian', 'border', '.', '[SEP]', 'kill', '[SEP]']


In [ ]:

task = 'SRL'
batch_size = 32
model_name = 'bert-base-uncased'

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)